In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from tqdm.auto import tqdm

C:\Users\Sam's PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
path = r"../data/raw/adFontes.csv"
df = pd.read_csv(path)
df['Text'] = ''
df['Swing'] = None

In [3]:
for i in range(df.shape[0]):
    if (df.at[i,'Bias']<=-6):
        df.at[i,'Swing'] = 'Left'
    elif (df.at[i,'Bias']>-6 and df.at[i,'Bias']<6):
        df.at[i,'Swing'] = 'Central'
    elif (df.at[i,'Bias']>=6):
        df.at[i,'Swing'] = 'Right'

In [4]:
print(df.columns)

Index(['Source', 'Url', 'Bias', 'Quality', 'Text', 'Swing'], dtype='object')


In [10]:
def Scrape(soup):
    body = ''
    body += BeautifulSoup(str(soup.find_all('p', {'class': ""})), 'html.parser').text
    if (body == 'None'):
        body += BeautifulSoup(str(soup.find_all('p')), 'html.parser').text
    if (body != '[]'  and body != ''):
        return body

In [12]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
for i in tqdm(range(df.shape[0])):
    try:
        session = requests.Session()
        session.headers = headers
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        page=session.get(str(df.iloc[i]['Url']))
        #print("Connection Sucsess on page " + str(i))
    except Exception:
        print(str(i)+" - error")
        continue
    soup = BeautifulSoup(page.text, 'html.parser')
    ret = Scrape(soup)
    if ret != False:
        #print(ret)
        df.at[i,"Text"] = ret

  0%|          | 0/1909 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df['Text'] = df['Text'].apply(lambda x: np.nan if x == ""  else x)
df['Text'] = df['Text'].apply(lambda x: np.nan if x == '[]' else x)
df['Text'] = df['Text'].apply(lambda x: np.nan if x == [] else x)

df.dropna(subset=['Text'],inplace=True)

In [ ]:
def Punctuation(string): 

    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
  
    return string

In [ ]:
df['Text'] = df['Text'].apply(lambda x:Punctuation(str(x)))

In [ ]:
path = r'backend/data/articles2.csv'
df.to_csv(path)